In [43]:
import pandas as pd
import operator
import time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [44]:
Final_Data = pd.read_csv('../teis_project-syndrome/data/teis_final_data.csv')

In [45]:
Final_Data.columns

Index(['Child ID', 'Notification Date', 'Fiscal Year', 'Notification Month',
       'Tenn Region', 'Fiscal Year.1', 'third_DOB', 'Late Referral', 'Qtr',
       'POE', 'DOB', 'County Name', 'County SES', 'Child Phase', 'Active',
       'Service Coordinator', 'Parent Consent', 'Initial Eligibility',
       'Initial Eligibility Date', 'Initial IFSP Date', 'Latest IFSP',
       'Exit Reason', 'Exit Date', 'Referral Source Type Name', 'Year'],
      dtype='object')

In [46]:
Final_Data.head(5)

,Child ID,Notification Date,Fiscal Year,Notification Month,Tenn Region,Fiscal Year.1,third_DOB,Late Referral,Qtr,POE,...,Service Coordinator,Parent Consent,Initial Eligibility,Initial Eligibility Date,Initial IFSP Date,Latest IFSP,Exit Reason,Exit Date,Referral Source Type Name,Year
0,104085,2018-01-20,2017-2018,Jan,Middle,2017.0,2010-07-07,1.0,3.0,GN,...,NaN,NaN,NaN,NaN,NaN,NaN,Unable to contact,2010-01-25,DCS,2018
1,350065,2016-08-17,2016-2017,Aug,Middle,2016.0,2016-10-15,1.0,1.0,UC,...,Steven Scarlett,NaN,NaN,NaN,NaN,NaN,Unable to contact,2016-08-31,PCP,2016
2,351755,2016-07-12,2016-2017,Jul,Western,2016.0,2016-11-13,NaN,1.0,NW,...,Rene Bard,2016-07-12,Ineligible,8/25/2016,NaN,NaN,Ineligible for Part C,2016-08-25,Parent,2016
3,353800,2016-09-14,2016-2017,Sep,Eastern,2016.0,2016-10-01,1.0,1.0,FT,...,Jennifer Terranera - 45 days,NaN,NaN,NaN,NaN,NaN,Referral less than 45 days,2016-09-15,Parent,2016
4,353805,2016-08-05,2016-2017,Aug,Middle,2016.0,2016-12-04,NaN,1.0,UC,...,Anna Bolin,2016-08-15,Eligible,10/5/2016,10/10/2016,NaN,618 - Part B eligibility not determined,2016-12-03,Dept. of Health,2016


In [47]:
Final_Data['Notification Date'] = pd.to_datetime(Final_Data['Notification Date'])
Final_Data['Notification Date']

0        2018-01-20
1        2016-08-17
2        2016-07-12
3        2016-09-14
4        2016-08-05
            ...    
115519   2022-07-01
115520   2022-07-01
115521   2022-07-01
115522   2022-07-01
115523   2022-07-01
Name: Notification Date, Length: 115524, dtype: datetime64[ns]

In [48]:
Final_Data['DOB'] = pd.to_datetime(Final_Data['DOB'])
Final_Data['DOB']

0        2007-07-07
1        2013-10-15
2        2013-11-13
3        2013-10-01
4        2013-12-04
            ...    
115519   2020-05-20
115520   2021-12-07
115521   2022-06-21
115522   2022-06-17
115523   2019-12-26
Name: DOB, Length: 115524, dtype: datetime64[ns]

In [49]:
TEIS_Exit = Final_Data[['POE', 'Service Coordinator', 'Child ID', 'DOB', 'third_DOB', 'Exit Reason','Exit Date','Child Phase']].fillna(0)
TEIS_Exit

,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase
0,GN,0,104085,2007-07-07,2010-07-07,Unable to contact,2010-01-25,Notification
1,UC,Steven Scarlett,350065,2013-10-15,2016-10-15,Unable to contact,2016-08-31,Screening
2,NW,Rene Bard,351755,2013-11-13,2016-11-13,Ineligible for Part C,2016-08-25,Eligibility
3,FT,Jennifer Terranera - 45 days,353800,2013-10-01,2016-10-01,Referral less than 45 days,2016-09-15,Notification
4,UC,Anna Bolin,353805,2013-12-04,2016-12-04,618 - Part B eligibility not determined,2016-12-03,IFSP
...,...,...,...,...,...,...,...,...
115519,GN,Kim Correll,531592,2020-05-20,0,0,0,Notification
115520,GN,Kim Correll,531593,2021-12-07,0,0,0,Notification
115521,GN,Kim Correll,531594,2022-06-21,0,0,0,Notification
115522,GN,Kim Correll,531595,2022-06-17,0,0,0,Notification


In [50]:
TEIS_Exit = TEIS_Exit[TEIS_Exit['Child Phase']== "IFSP"]
TEIS_Exit

,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase
4,UC,Anna Bolin,353805,2013-12-04,2016-12-04,618 - Part B eligibility not determined,2016-12-03,IFSP
8,ET,Kristi Borer,354276,2013-12-24,2016-12-24,618 - Parent withdraw,2016-09-22,IFSP
15,ET,Karen Stock,358011,2014-02-02,2017-02-02,618 - Part B eligible,2017-02-01,IFSP
16,FT,Jennifer Terranera,358031,2014-01-14,2017-01-14,618 - Part B eligible,2017-01-13,IFSP
18,FT,Kathy Jeffries,358765,2014-02-04,2017-02-04,618 - Part B eligible,2017-02-03,IFSP
...,...,...,...,...,...,...,...,...
114258,NW,Brittany Smith,530139,2020-05-06,0,0,0,IFSP
114296,GN,Katrina Mangrum,530190,2020-12-01,0,0,0,IFSP
114301,SC,Andrea Smith,530196,2020-06-06,0,0,0,IFSP
114319,UC,Candace Peacock,530214,2021-01-12,0,0,0,IFSP


In [51]:
TEIS_Exit = TEIS_Exit[TEIS_Exit['Exit Date'] != 0]
TEIS_Exit

,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase
4,UC,Anna Bolin,353805,2013-12-04,2016-12-04,618 - Part B eligibility not determined,2016-12-03,IFSP
8,ET,Kristi Borer,354276,2013-12-24,2016-12-24,618 - Parent withdraw,2016-09-22,IFSP
15,ET,Karen Stock,358011,2014-02-02,2017-02-02,618 - Part B eligible,2017-02-01,IFSP
16,FT,Jennifer Terranera,358031,2014-01-14,2017-01-14,618 - Part B eligible,2017-01-13,IFSP
18,FT,Kathy Jeffries,358765,2014-02-04,2017-02-04,618 - Part B eligible,2017-02-03,IFSP
...,...,...,...,...,...,...,...,...
112126,SC,Kelly Holt,527606,2021-12-09,2024-12-09,618 - Moved out of state,2022-06-27,IFSP
112129,MD,Torkwase Smith,527609,2019-08-26,2022-08-26,618 - Parent withdraw,2022-06-13,IFSP
112174,SC,Brittney Blalock Rose,527688,2020-04-24,2023-04-24,618 - Parent withdraw,2022-06-08,IFSP
112689,MD,Marian Kingery,528318,2021-09-26,2024-09-26,618 - Attempts to contact unsuccessful,2022-07-01,IFSP


In [52]:
TEIS_Exit['third_DOB'] = pd.to_datetime(TEIS_Exit['third_DOB'])
TEIS_Exit['third_DOB']

C:\Users\marti\AppData\Local\Temp\ipykernel_20804\2100382324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEIS_Exit['third_DOB'] = pd.to_datetime(TEIS_Exit['third_DOB'])


4        2016-12-04
8        2016-12-24
15       2017-02-02
16       2017-01-14
18       2017-02-04
            ...    
112126   2024-12-09
112129   2022-08-26
112174   2023-04-24
112689   2024-09-26
113003   2023-10-11
Name: third_DOB, Length: 36040, dtype: datetime64[ns]

In [53]:
TEIS_Exit["Exit Date"] = pd.to_datetime(TEIS_Exit['Exit Date'])

C:\Users\marti\AppData\Local\Temp\ipykernel_20804\1525979668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEIS_Exit["Exit Date"] = pd.to_datetime(TEIS_Exit['Exit Date'])


In [54]:
TEIS_Exit["Age of Exit"] = TEIS_Exit["Exit Date"] - TEIS_Exit["DOB"]
TEIS_Exit

C:\Users\marti\AppData\Local\Temp\ipykernel_20804\2633957898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEIS_Exit["Age of Exit"] = TEIS_Exit["Exit Date"] - TEIS_Exit["DOB"]


,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase,Age of Exit
4,UC,Anna Bolin,353805,2013-12-04,2016-12-04,618 - Part B eligibility not determined,2016-12-03,IFSP,1095 days
8,ET,Kristi Borer,354276,2013-12-24,2016-12-24,618 - Parent withdraw,2016-09-22,IFSP,1003 days
15,ET,Karen Stock,358011,2014-02-02,2017-02-02,618 - Part B eligible,2017-02-01,IFSP,1095 days
16,FT,Jennifer Terranera,358031,2014-01-14,2017-01-14,618 - Part B eligible,2017-01-13,IFSP,1095 days
18,FT,Kathy Jeffries,358765,2014-02-04,2017-02-04,618 - Part B eligible,2017-02-03,IFSP,1095 days
...,...,...,...,...,...,...,...,...,...
112126,SC,Kelly Holt,527606,2021-12-09,2024-12-09,618 - Moved out of state,2022-06-27,IFSP,200 days
112129,MD,Torkwase Smith,527609,2019-08-26,2022-08-26,618 - Parent withdraw,2022-06-13,IFSP,1022 days
112174,SC,Brittney Blalock Rose,527688,2020-04-24,2023-04-24,618 - Parent withdraw,2022-06-08,IFSP,775 days
112689,MD,Marian Kingery,528318,2021-09-26,2024-09-26,618 - Attempts to contact unsuccessful,2022-07-01,IFSP,278 days


In [55]:
TEIS_Exit["Age of Exit"] = TEIS_Exit["Age of Exit"].dt.days.astype(int)

C:\Users\marti\AppData\Local\Temp\ipykernel_20804\246842160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEIS_Exit["Age of Exit"] = TEIS_Exit["Age of Exit"].dt.days.astype(int)


In [56]:
TEIS_Exit = TEIS_Exit[TEIS_Exit['Age of Exit']  < 1095]
TEIS_Exit

,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase,Age of Exit
8,ET,Kristi Borer,354276,2013-12-24,2016-12-24,618 - Parent withdraw,2016-09-22,IFSP,1003
22,MD,Karen Hunter,360748,2014-01-29,2017-01-29,618 - Part B eligibility not determined,2017-01-27,IFSP,1094
52,GN,Jennifer Johnson,364743,2014-03-13,2017-03-13,618 - Parent withdraw,2016-07-08,IFSP,848
57,ET,Cathy Taylor,365848,2014-04-03,2017-04-03,618 - Attempts to contact unsuccessful,2016-10-12,IFSP,923
67,SW,Elizabeth Hailey,368391,2014-06-15,2017-06-15,618 - Attempts to contact unsuccessful,2017-05-23,IFSP,1073
...,...,...,...,...,...,...,...,...,...
112126,SC,Kelly Holt,527606,2021-12-09,2024-12-09,618 - Moved out of state,2022-06-27,IFSP,200
112129,MD,Torkwase Smith,527609,2019-08-26,2022-08-26,618 - Parent withdraw,2022-06-13,IFSP,1022
112174,SC,Brittney Blalock Rose,527688,2020-04-24,2023-04-24,618 - Parent withdraw,2022-06-08,IFSP,775
112689,MD,Marian Kingery,528318,2021-09-26,2024-09-26,618 - Attempts to contact unsuccessful,2022-07-01,IFSP,278


In [61]:
TEIS_Check = TEIS_Exit[TEIS_Exit['Age of Exit']  == 1094]
TEIS_Check.tail(30)

,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase,Age of Exit
93114,GN,Pamela Bouchard-SC,502642,2019-05-15,2022-05-15,618 - Part B eligibility not determined,2022-05-13,IFSP,1094
93338,GN,Jessica Shook,502914,2019-04-03,2022-04-03,618 - Part B eligibility not determined,2022-04-01,IFSP,1094
93769,GN,Tina Regenwether,503399,2019-03-06,2022-03-06,618 - Part B eligibility not determined,2022-03-04,IFSP,1094
94213,GN,Tina Regenwether,503943,2018-12-24,2021-12-24,618 - Part B eligibility not determined,2021-12-22,IFSP,1094
94607,FT,Kimberly Johnson,504429,2019-05-15,2022-05-15,618 - Not eligible for Part B – Exit with refe...,2022-05-13,IFSP,1094
94872,ET,Andrea Mathews,504724,2019-02-13,2022-02-13,618 - Part B eligible,2022-02-11,IFSP,1094
95650,GN,Laurie Mitchell,505570,2018-11-07,2021-11-07,618 - Part B eligibility not determined,2021-11-05,IFSP,1094
95898,GN,Merissa Leavitt,505842,2019-04-24,2022-04-24,618 - Part B eligibility not determined,2022-04-22,IFSP,1094
96084,FT,Rebecca Taylor,506008,2019-05-01,2022-05-01,618 - Part B eligibility not determined,2022-04-29,IFSP,1094
97225,GN,Merissa Leavitt,507228,2019-05-08,2022-05-08,618 - Part B eligibility not determined,2022-05-06,IFSP,1094


In [64]:
TEIS_Check2 = TEIS_Exit[TEIS_Exit['Age of Exit']  == 1093]
TEIS_Check2.tail(40)

,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase,Age of Exit
89443,GN,Marguarite Stewart,498796,2019-06-27,2022-06-27,618 - Part B eligible,2022-06-24,IFSP,1093
89643,GN,Marisa Nicklaus,498995,2019-03-21,2022-03-21,618 - Part B eligible,2022-03-18,IFSP,1093
89817,FT,Kathy Jeffries,499208,2018-07-05,2021-07-05,618 - Not eligible for Part B – Exit with refe...,2021-07-02,IFSP,1093
90193,GN,Marisa Nicklaus,499628,2019-05-09,2022-05-09,618 - Part B eligibility not determined,2022-05-06,IFSP,1093
90300,GN,Jessica Shook,499748,2019-05-09,2022-05-09,618 - Part B eligible,2022-05-06,IFSP,1093
90341,GN,Jessica Shook,499798,2019-05-30,2022-05-30,618 - Part B eligibility not determined,2022-05-27,IFSP,1093
90364,FT,Alicia Taylor,499827,2018-09-13,2021-09-13,618 - Part B eligibility not determined,2021-09-10,IFSP,1093
90371,GN,Jessica Shook,499833,2018-09-13,2021-09-13,618 - Part B eligibility not determined,2021-09-10,IFSP,1093
90584,UC,Miriam Espinosa,500035,2019-03-07,2022-03-07,618 - Not eligible for Part B – Exit with refe...,2022-03-04,IFSP,1093
90980,MD,Shannon Denley,500459,2019-04-25,2022-04-25,618 - Parent withdraw,2022-04-22,IFSP,1093


In [63]:
TEIS_Check3 = TEIS_Exit[TEIS_Exit['Age of Exit']  == 1092]
TEIS_Check3.tail(30)

,POE,Service Coordinator,Child ID,DOB,third_DOB,Exit Reason,Exit Date,Child Phase,Age of Exit
77254,GN,Kira Nemeth,486263,2018-07-06,2021-07-06,618 - Part B eligibility not determined,2021-07-02,IFSP,1092
77546,GN,Alexis Seaton,486574,2017-11-29,2020-11-29,618 - Part B eligibility not determined,2020-11-25,IFSP,1092
78107,GN,Katrina Mangrum,487223,2018-02-16,2021-02-16,618 - Part B eligible,2021-02-12,IFSP,1092
78964,GN,Chris Foy,488066,2017-11-29,2020-11-29,618 - Part B eligibility not determined,2020-11-25,IFSP,1092
79433,SW,Katie Gentry,488547,2019-05-28,2022-05-28,618 - Parent withdraw,2022-05-24,IFSP,1092
79943,GN,Mary Jane Ratliff,489037,2018-09-07,2021-09-07,618 - Part B eligible,2021-09-03,IFSP,1092
81671,GN,Kenya Bass,490859,2018-09-07,2021-09-07,618 - Part B eligibility not determined,2021-09-03,IFSP,1092
82479,SC,Tracey Roberson,491639,2018-07-16,2021-07-16,618 - Parent withdraw,2021-07-12,IFSP,1092
82675,GN,Kenya Bass,491886,2018-12-25,2021-12-25,618 - Part B eligibility not determined,2021-12-21,IFSP,1092
82679,GN,Chris Foy,491890,2019-04-18,2022-04-18,618 - Part B eligibility not determined,2022-04-14,IFSP,1092


In [ ]:
TEIS_Exit = TEIS_Exit[TEIS_Exit['Age of Exit']  < 1092]
TEIS_Exit

TEIS_Exit = TEIS_Exit[TEIS_Exit['Exit Reason']  != "618 - Deceased"]
TEIS_Exit

TEIS_Exit = TEIS_Exit[TEIS_Exit['Exit Reason']  != "618 - Completion of IFSP/ no longer requires services"]
TEIS_Exit

TEIS_Exit = TEIS_Exit[TEIS_Exit['Exit Reason']  != "618 - Moved out of state"]
TEIS_Exit

TEIS_Exit_Poe = TEIS_Exit.groupby("POE").count()["Age of Exit"]
TEIS_Exit_Poe



TEIS_Exit_Poe = TEIS_Exit.groupby("POE").count()["Age of Exit"]
TEIS_Exit_Poe = TEIS_Exit_Poe.to_frame()
TEIS_Exit_Poe


TEIS_Exit_Poe_merge = pd.merge(TEIS_Exit_Poe, TEIS_All_Exits, on = ['POE'], how = 'inner')
TEIS_Exit_Poe_merge['Rate of Exit'] = TEIS_Exit_Poe_merge['Age of Exit']/TEIS_Exit_Poe_merge['Child ID']*100
TEIS_Exit_Poe_merge

TEIS_Exit_SC = TEIS_Exit.groupby("Service Coordinator").count()["Age of Exit"]
TEIS_Exit_SC = TEIS_Exit_SC.to_frame()
TEIS_Exit_SC




TEIS_Exit_SC_merge2 = pd.merge(TEIS_Exit_SC, TEIS_All_Exits2, on = ['Service Coordinator'], how = 'inner')
TEIS_Exit_SC_merge2['Rate of Exit'] = TEIS_Exit_SC_merge2['Age of Exit']/TEIS_Exit_SC_merge2['Child ID']*100
TEIS_Exit_SC_merge2.sort_values(by = "Rate of Exit")